In [ ]:
import pandas as pd

raw_data = pd.read_csv(r'.\Datasets\data.csv')
raw_data.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(np.array(raw_data['zip_code']).reshape(-1, 1))
new_cols = pd.DataFrame(encoder
                         .transform(np.array(raw_data['zip_code'])
                                    .reshape(-1, 1))
                         .toarray().astype('int64'),
                         columns=encoder.categories_[0])
raw_data = raw_data.join(new_cols)

In [ ]:
raw_data['Phone'] = raw_data.channel.apply(lambda x: 1 if x != 'Web' else 0)
raw_data['Web'] = raw_data.channel.apply(lambda x: 1 if x != 'Phone' else 0)

In [ ]:
raw_data['treatment'] = raw_data.offer.apply(lambda x: 1 if x != 'No Offer' else 0)

In [ ]:
converted = raw_data[raw_data['conversion'] == 1].copy().reset_index()
converted.info()

In [ ]:
import matplotlib.pyplot as plt

plt.title('Proporção entre clientes convertidos e não convertidos')
raw_data['conversion'].value_counts().plot(kind='bar')
print(raw_data['conversion'].value_counts())

In [ ]:
plt.title(Proporção de clientes que receberam ou não uma oferta')
raw_data[treatment].value_counts().plot(kind='bar')
print(raw_data[treatment].value_counts())

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(confusion_matrix(raw_data['treatment'], raw_data['conversion']))

In [ ]:
print(f'Receita total: $ {raw_data.history.sum():,.2f}')
print(f'Receita de clientes convertidos: $ {converted.history.sum():,.2f}')

In [ ]:
from scipy.stats import ttest_ind

df_control = raw_data[raw_data.treatment==0]
df_treatment = raw_data[raw_data.treatment==1]

print(f'média controle: {df_control.conversion.mean()}',
        f'média tratamento: {df_treatment.conversion.mean()}')

# comparando amostras
_, p = ttest_ind(df_control.conversion, df_treatment.conversion)
print(f'p={p:.3f}')

# interpretação
alpha = 0.05  # nível de significância
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')

In [ ]:
print(f'média controle: {df_control.history.mean()}',
        f'média tratamento: {df_treatment.history.mean()}')

# comparando amostras
_, p = ttest_ind(df_control.history, df_treatment.history)
print(f'p={p:.3f}')

# interpretação
alpha = 0.05  # nível de significância
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')

In [ ]:
import numpy as np

correlation = raw_data.corr()

mask = np.zeros_like(correlation, dtype=bool)
mask[np.triu_indices_from(mask)] = True
correlation[mask] = np.nan

correlation \
    .style \
    .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1) \
    .highlight_null(null_color='#f1f1f1') \
    .set_precision(2)

In [ ]:
from sklearn.linear_model import LogisticRegression

X = raw_data.drop(['zip_code', 'channel', 'offer', 'treatment', 'conversion'], axis=1)
y = raw_data['treatment']

lr = LogisticRegression(max_iter=1000)
lr.fit(X, y)

coeffs = pd.DataFrame({
    'column': X.columns.to_numpy(),
    'coeff': lr.coef_.ravel()
})

coeffs

In [ ]:
import math

pred_bin = lr.predict(X)
pred_prob = lr.predict_proba(X)

def logit(p):
    logit_value = math.log(p / (1 - p))
    return logit_value

raw_data['ps'] = pred_prob[:, 1]
raw_data['ps_logit'] = raw_data.ps.apply(lambda x: logit(x))

sns.histplot(data=raw_data, x='ps', hue='treatment');

In [ ]:
caliper = np.std(raw_data.ps) * 0.25
print(f'calibre = {caliper:.4f}')

n_neighbors = 10

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, radius=caliper)
ps = raw_data[['ps']]
knn.fit(ps)

In [ ]:
distances, neighbor_indexes = knn.kneighbors(ps)

matched_control = []
for current_index, row in raw_data.iterrows():
    if row.treatment == 0: 
        raw_data.loc[current_index, 'matched'] = np.nan
    else: 
        for idx in neighbor_indexes[current_index, :]:
            if (current_index != idx) and (raw_data.loc[idx].treatment == 0):
                if idx not in matched_control:
                    raw_data.loc[current_index, 'matched'] = idx 
                    matched_control.append(idx)
                    break

print('observações totais no grupo de abordados: ', len(raw_data[raw_data.treatment == 1]))
print('observações totais no grupo controle: ', len(matched_control))

In [ ]:
treatment_matched = raw_data.dropna(subset=['matched'])

control_matched_idx = treatment_matched.matched
control_matched_idx = control_matched_idx.astype(int) 
control_matched = raw_data.loc[control_matched_idx, :] 

df_matched = pd.concat([treatment_matched, control_matched])

df_matched.treatment.value_counts()

In [ ]:
print(f'média controle: {df_matched_control.conversion.mean()}',
        f'média tratamento: {df_matched_treatment.conversion.mean()}')

# comparando amostras
_, p = ttest_ind(df_matched_control.conversion, df_matched_treatment.conversion)
print(f'p={p:.3f}')

# interpretação
alpha = 0.05  # nível de significância
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')

In [ ]:
df_matched_control = df_matched[df_matched.treatment==0]
df_matched_treatment = df_matched[df_matched.treatment==1]

print(f'média controle {df_matched_control.history.mean()}',
      f'média tratamento {df_matched_treatment.history.mean()}')

# comparando amostras
_, p = ttest_ind(df_matched_control.history, df_matched_treatment.history)
print(f'p={p:.3f}')

# interpretação
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=30, radius=caliper)
ps = raw_data[['ps']]
knn.fit(ps)

In [ ]:
distances, neighbor_indexes = knn.kneighbors(ps)

matched_control = []
for current_index, row in raw_data.iterrows():
    if row.treatment == 0: 
        raw_data.loc[current_index, 'matched'] = np.nan
    else: 
        for idx in neighbor_indexes[current_index, :]:
            if (current_index != idx) and (raw_data.loc[idx].treatment == 0):
                if idx not in matched_control:
                    raw_data.loc[current_index, 'matched'] = idx 
                    matched_control.append(idx)
                    break

print('observações totais no grupo de abordados: ', len(raw_data[raw_data.treatment == 1]))
print('observações totais no grupo controle: ', len(matched_control))

In [ ]:
treatment_matched = raw_data.dropna(subset=['matched'])

control_matched_idx = treatment_matched.matched
control_matched_idx = control_matched_idx.astype(int) 
control_matched = raw_data.loc[control_matched_idx, :] 

df_matched = pd.concat([treatment_matched, control_matched])

df_matched.treatment.value_counts()

In [ ]:
print(f'média controle: {df_matched_control.conversion.mean()}',
        f'média tratamento: {df_matched_treatment.conversion.mean()}')

# comparando amostras
_, p = ttest_ind(df_matched_control.conversion, df_matched_treatment.conversion)
print(f'p={p:.3f}')

# interpretação
alpha = 0.05  # nível de significância
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')

In [ ]:
df_matched_control = df_matched[df_matched.treatment==0]
df_matched_treatment = df_matched[df_matched.treatment==1]

print(f'média controle {df_matched_control.history.mean()}',
      f'média tratamento {df_matched_treatment.history.mean()}')

# comparando amostras
_, p = ttest_ind(df_matched_control.history, df_matched_treatment.history)
print(f'p={p:.3f}')

# interpretação
if p > alpha:
    print('sem evidências suficientes para rejeitar H0 - não se pode afirmar que as médias são estatisticamente diferentes')
else:
    print('H0 rejeitado - médias são estatisticamente diferentes')